In [24]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import coint


In [25]:
# Get the data from tsm and qcom with inteval of 30 minutes

tsm = yf.Ticker("TSM")
qcom = yf.Ticker("QCOM")

tsm_data = tsm.history(period="60d", interval="30m")
qcom_data = qcom.history(period="60d", interval="30m")

In [26]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close'], name='QCOM'))

fig.show()

In [27]:
print(tsm_data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-02-22 09:30:00-05:00  129.610001  131.289993  129.259995  130.882095   
2024-02-22 10:00:00-05:00  130.882095  131.070007  130.070007  130.979996   
2024-02-22 10:30:00-05:00  130.977005  131.479996  130.690002  131.029999   
2024-02-22 11:00:00-05:00  131.029999  131.049896  130.100006  130.309998   
2024-02-22 11:30:00-05:00  130.350006  130.899994  130.322495  130.779999   
...                               ...         ...         ...         ...   
2024-05-16 09:30:00-04:00  154.100006  155.054993  153.600006  154.610001   
2024-05-16 10:00:00-04:00  154.429993  154.649994  153.600006  154.485001   
2024-05-16 10:30:00-04:00  154.475006  154.600006  154.100006  154.500000   
2024-05-16 11:00:00-04:00  154.509995  154.710007  154.110001  154.649902   
2024-05-16 11:30:00-04:00  154.649994  154.699997  154.619995  154.660004   

In [28]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [29]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

tsm_data['5T_MA'] = tsm_data['Close'].rolling(window=5).mean()
tsm_data['20T_MA'] = tsm_data['Close'].rolling(window=20).mean()

tsm_data['signal'] = np.where(tsm_data['5T_MA'] > tsm_data['20T_MA'], 1, 0)
tsm_data['signal'] = tsm_data['signal'].diff()


print(tsm_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    725
 1.0     23
-1.0     23
Name: count, dtype: int64


In [30]:
# calculate profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

tsm_data['buy_price'] = tsm_data['buy_price'].ffill()
tsm_data['sell_price'] = tsm_data['sell_price'].ffill()

tsm_data['PnL'] = tsm_data['sell_price'] - tsm_data['buy_price']

print(tsm_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

-511.47864532470703


In [31]:
ratio_0 = (tsm_data['Close'].iloc[0]) / (qcom_data['Close'].iloc[0])
ratio_0

0.8575122266254513

In [32]:
qcom_data['Close_NORM'] = qcom_data['Close'].multiply(ratio_0).round(6)

In [33]:
tsm_qcom_dif = tsm_data['Close'] - qcom_data['Close_NORM']
tsm_qcom_dif

Datetime
2024-02-22 09:30:00-05:00    3.369141e-07
2024-02-22 10:00:00-05:00   -6.095443e-01
2024-02-22 10:30:00-05:00   -3.408722e-01
2024-02-22 11:00:00-05:00   -1.347033e+00
2024-02-22 11:30:00-05:00   -1.371211e+00
                                 ...     
2024-05-16 09:30:00-04:00   -1.275066e+01
2024-05-16 10:00:00-04:00   -1.167515e+01
2024-05-16 10:30:00-04:00   -1.184451e+01
2024-05-16 11:00:00-04:00   -1.205905e+01
2024-05-16 11:30:00-04:00   -1.204895e+01
Length: 772, dtype: float64

In [34]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))

fig.show()

In [35]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close_NORM'], name='QCOM'))

fig.show()

In [36]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [37]:
tsm_qcom_dif = tsm_qcom_dif.to_frame()
tsm_qcom_dif.columns = ['Close']

In [38]:
tsm_qcom_dif['10T_MA'] = tsm_qcom_dif['Close'].rolling(window=10).mean()
tsm_qcom_dif['60T_MA'] = tsm_qcom_dif['Close'].rolling(window=60).mean()

tsm_qcom_dif['signal'] = np.where(tsm_qcom_dif['10T_MA'] > tsm_qcom_dif['60T_MA'], -1, 0)
tsm_qcom_dif['signal'] = tsm_qcom_dif['signal'].diff()

print(tsm_qcom_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=60).mean()['Close'], name='60 Ticks MA'))

buy_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == 1]
sell_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # -1 buy || 1 sell

tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal'] # opposite signal

fig.update_layout(title='TSM-QCOM Moving Average Crossover Strategy (Short Average = 10, Long Average = 60) (60 days)')

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal'] == 1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal'] == -1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


signal
 0.0    755
-1.0      8
 1.0      8
Name: count, dtype: int64


TSMC Profit =>  -5.32720947265625
QCOM Profit =>  15.91379199999983
Pair profit =>  10.58658252734358


In [39]:
#Consider TSM as the first and QCOM as the second ticker.
#When there is a buy signal for TSM, we buy TSM and sell QCOM.
#When there is a sell signal for TSM, we sell TSM and buy QCOM.

In [40]:
#Copy signal column from difference dataframe to TSM dataframe
tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal']

In [41]:
tsm_data.reset_index(inplace=True)
qcom_data.reset_index(inplace=True)

# Convert datetime columns to indexes
tsm_data.set_index('Datetime', inplace=True)
qcom_data.set_index('Datetime', inplace=True)

# Find common datetime values
common_dates = tsm_data.index.intersection(qcom_data.index)

# Align the data and handle missing values
tsm_close_aligned = tsm_data.loc[common_dates, 'Close']
qcom_close_aligned = qcom_data.loc[common_dates, 'Close_NORM']

# Perform cointegration test
cointegration_test_result = coint(tsm_close_aligned, qcom_close_aligned)


# Print the result
print("Cointegration Test Result:")
print("Test statistic:", cointegration_test_result[0])
print("p-value:", cointegration_test_result[1])
print("Critical values:", cointegration_test_result[2])



Cointegration Test Result:
Test statistic: -2.9768131274307676
p-value: 0.1158203916304884
Critical values: [-3.9107012  -3.34406638 -3.04995548]


In [42]:
# Calculate the z score of the difference
tsm_qcom_dif['z_score'] = (tsm_qcom_dif['Close'] - tsm_qcom_dif['Close'].rolling(window=20).mean()) / tsm_qcom_dif['Close'].rolling(window=20).std()

In [43]:
# function to implement a flag that is (-1 sell order open, 0 no order open, 1 buy order open) to control the signal_z_score signals

def restrict_signals(dataset, signal_col, flag_col):
    dataset[flag_col] = 0
    # cicle to put flags
    for i in range(dataset.shape[0]):
        if dataset[signal_col].iloc[i] == 1:
            if dataset[flag_col].iloc[i] < 1:
                dataset[flag_col].iloc[i] += 1
                print('Buy signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset buy signal')
        elif dataset[signal_col].iloc[i] == -1:
            if dataset[flag_col].iloc[i] > -1:
                dataset[flag_col].iloc[i] += -1
                print('Sell signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset sell signal')

    return dataset
        

In [44]:
# plot the z score along with the buy and sell signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['z_score'], name='Z Score'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[2] * len(tsm_qcom_dif), name='Upper Threshold'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[-2] * len(tsm_qcom_dif), name='Lower Threshold'))

#buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] < -2]
#sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] > 2]

# TODO: check this
#tsm_qcom_dif['signal_z_score'] = np.where(((tsm_qcom_dif['z_score'] < -2) and (tsm_qcom_dif['z_score_sig_flag'] < 1)), 1, 0) + np.where(((tsm_qcom_dif['z_score'] > 2) and (tsm_qcom_dif['z_score_sig_flag'] > 1)), -1, 0)
tsm_qcom_dif['signal_z_score'] = np.where((tsm_qcom_dif['z_score'] < -2), 1, 0) + np.where((tsm_qcom_dif['z_score'] > 2), -1, 0)

#tsm_qcom_dif = restrict_signals(tsm_qcom_dif, 'signal_z_score', 'z_score_sig_flag')

""" INIT """
flag_signal_z_score = 0

for i in range(tsm_qcom_dif.shape[0]):
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    if tsm_qcom_dif['signal_z_score'].iloc[i] == 1:
        print('in', flag_signal_z_score)
        if flag_signal_z_score < 1:
            flag_signal_z_score = 1
            print('Buy signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset buy signal')
    elif tsm_qcom_dif['signal_z_score'].iloc[i] == -1:
        if flag_signal_z_score > -1:
            flag_signal_z_score = -1
            print('Sell signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset sell signal')
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    

""" END """
# plot the signals
buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1][:-1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score.index, y=buy_signals_z_score['z_score'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score.index, y=sell_signals_z_score['z_score'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.update_layout(title='Z Score with Buy/Sell Signals (60 days)')

fig.show()


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 0
Buy signal
1
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal
0
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Reset sell signal
0
0
0
-1
Reset sell signal
0
0
0
0
0
0
0
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Reset sell signal
0
-1
Reset sell signal
0
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

/tmp/ipykernel_148945/2787702362.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_148945/2787702362.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_148945/2787702362.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_148945/2787702362.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

## Calculate Profit using Z-score

In [45]:
# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))

buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1][:-1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score.index, y=buy_signals_z_score['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score.index, y=sell_signals_z_score['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # 1 buy || - 1 sell

tsm_data['signal_z_score'] = tsm_qcom_dif['signal_z_score']
qcom_data['signal_z_score'] = - tsm_qcom_dif['signal_z_score'] # opposite signal

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal_z_score'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal_z_score'] == -1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal_z_score'] == 1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal_z_score'] == -1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


TSMC Profit =>  -133.37689208984375
QCOM Profit =>  155.73468200000002
Pair profit =>  22.35778991015627


In [46]:
#Testar Métodos de Mercados Diferentes, Mercados Tecnológicos
# A cointegração muda com 